In [1]:
from fastai.text.all import *

# Neural Networks for Sequences

- input is a sequence
- ouput is a sequence

A model that has been trained to guess the next word in a text. This is a *self-supervised learning* (no labels are required for learning). 

## Word Tokenization

In [2]:
spacy = WordTokenizer()

## Natural language Processing

## Training a Text Classifier

## Recurrent Neural Networks (RNN)